### Link predictions - MA

1. Basic link prediction in the log space (Use MA first)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [77]:
import statsmodels

In [3]:
# read files
flow_16_acs_ma=gpd.read_file('../../data/processed_small_data/flow_16_acs_MA_shp.geojson')
flow_20_march_week4_ma=gpd.read_file('../../data/processed_small_data/flow_20_march_week4_MA_shp.geojson')
us_tract_ma=gpd.read_file('../../data/processed_small_data/us_tract_MA_shp.geojson')

In [8]:
# 
print(flow_16_acs_ma.shape)
print(flow_20_march_week4_ma.shape)
print(us_tract_ma.shape)

(101633, 13)
(63848, 10)
(1459, 66)


In [30]:
# 
us_tract_ma = us_tract_ma.to_crs('EPSG:5070')

In [73]:
flow_16_acs_ma.head()

,Ofips,Dfips,Ostfips,Octfips,Otrfips,Dstfips,Dctfips,Dtrfips,Flow,Moe,LenKM,ESTDIVMOE,geometry
0,25015820400,25015820400,25,15,820400,25,15,820400,2505,391,0.0,6.41,"LINESTRING (1900558.820 2390415.541, 1900558.8..."
1,25003925100,25003925100,25,3,925100,25,3,925100,2050,317,0.0,6.47,"LINESTRING (1840991.191 2354932.119, 1840991.1..."
2,25021404400,25021404400,25,21,404400,25,21,404400,1915,390,0.0,4.91,"LINESTRING (2000840.403 2404838.904, 2000840.4..."
3,25001010100,25001010100,25,1,10100,25,1,10100,1540,246,0.0,6.26,"LINESTRING (2094693.021 2403721.152, 2094693.0..."
4,25003914100,25003914100,25,3,914100,25,3,914100,1215,256,0.0,4.75,"LINESTRING (1847368.343 2366845.098, 1847368.3..."


In [10]:
flow_20_march_week4_ma.head()

,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date_range,visitor_flows,pop_flows,geometry
0,25001012700,25017342400,-70.341803,41.645342,-71.062198,42.398850,03/23/20 - 03/29/20,4,109.0,"LINESTRING (2095835.909 2355892.236, 2016504.4..."
1,25001012700,25001015300,-70.341803,41.645342,-70.290085,41.673588,03/23/20 - 03/29/20,215,5883.0,"LINESTRING (2095835.909 2355892.236, 2099112.2..."
2,25001012700,25001010400,-70.341803,41.645342,-69.983433,41.757063,03/23/20 - 03/29/20,20,547.0,"LINESTRING (2095835.909 2355892.236, 2120999.0..."
3,25001012700,25023545100,-70.341803,41.645342,-70.754009,41.768764,03/23/20 - 03/29/20,4,109.0,"LINESTRING (2095835.909 2355892.236, 2059342.3..."
4,25001012700,25025081001,-70.341803,41.645342,-71.106830,42.334567,03/23/20 - 03/29/20,4,109.0,"LINESTRING (2095835.909 2355892.236, 2014839.6..."


In [11]:
us_tract_ma.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,hh_worker_2_more,pct_worker_2_more,binge,bphigh,chd,diabetes,obesity,lpa,total_pop_cdc,geometry
0,25,027,724100,25027724100,7241,Census Tract 7241,G5020,S,53034202,1639215,...,577,37.54,18.0,33.5,6.9,9.4,30.7,25.3,3701,"POLYGON ((-72.21782 42.27018, -72.21765 42.270..."
1,25,027,759100,25027759100,7591,Census Tract 7591,G5020,S,25570219,1427831,...,358,43.34,20.5,28.9,5.8,8.3,31.1,23.4,2183,"POLYGON ((-72.07888 42.21475, -72.07880 42.214..."
2,25,025,091300,25025091300,913,Census Tract 913,G5020,S,262560,0,...,316,37.00,18.8,29.1,6.2,11.6,28.0,32.3,2499,"POLYGON ((-71.07220 42.32136, -71.07174 42.321..."
3,25,025,110501,25025110501,1105.01,Census Tract 1105.01,G5020,S,647837,0,...,623,45.67,22.8,24.8,4.8,7.1,20.6,19.2,3253,"POLYGON ((-71.14777 42.28599, -71.14739 42.286..."
4,25,025,030500,25025030500,305,Census Tract 305,G5020,S,193673,292691,...,481,31.83,29.1,16.0,2.8,3.8,15.5,12.5,2538,"POLYGON ((-71.05979 42.36877, -71.05883 42.369..."


# Preprocessing flow data

In [72]:
# pre processing flow_20
flow_20_march_week4_ma_all = flow_20_march_week4_ma.merge(us_tract_ma, left_on='geoid_o', right_on='GEOID', how='inner', suffixes=('_od', '_o'))
flow_20_march_week4_ma_all = flow_20_march_week4_ma_all.merge(us_tract_ma, left_on='geoid_d', right_on='GEOID', how='inner', suffixes=('_o', '_d'))
flow_20_march_week4_ma_all.head()
print("All columns are: ", [col for col in flow_20_march_week4_ma_all.columns])
print("Data shape (pre filering): ", flow_20_march_week4_ma_all.shape)

# turn to gpd with crs
flow_20_march_week4_ma_all = gpd.GeoDataFrame(flow_20_march_week4_ma_all, geometry='geometry_od',crs='EPSG:5070')

# add Euclidean distances & filter values
flow_20_march_week4_ma_all['euclidean_distance'] = flow_20_march_week4_ma_all.length
# filtering the self-flows
flow_20_march_week4_ma_all = flow_20_march_week4_ma_all.loc[flow_20_march_week4_ma_all['euclidean_distance']>0.0, :]
print("Data shape (filtering the self-loops)", flow_20_march_week4_ma_all.shape)

# drop ALL na
np.sum(flow_20_march_week4_ma_all.isna())
flow_20_march_week4_ma_all.dropna(inplace = True, how = 'any')
print("Data shape (after dropping all NAs)", flow_20_march_week4_ma_all.shape)


All columns are:  ['geoid_o', 'geoid_d', 'lng_o', 'lat_o', 'lng_d', 'lat_d', 'date_range', 'visitor_flows', 'pop_flows', 'geometry_od', 'STATEFP_o', 'COUNTYFP_o', 'TRACTCE_o', 'GEOID_o', 'NAME_o', 'NAMELSAD_o', 'MTFCC_o', 'FUNCSTAT_o', 'ALAND_o', 'AWATER_o', 'INTPTLAT_o', 'INTPTLON_o', 'layer_o', 'path_o', 'geoid_x_o', 'pop_o', 'households_o', 'male_o', 'male_ratio_o', 'female_o', 'female_ratio_o', 'median_age_o', 'white_o', 'black_o', 'asian_o', 'other_race_o', 'white_ratio_o', 'black_ratio_o', 'asian_ratio_o', 'other_race_ratio_o', 'income_o', 'pop_no_inc_o', 'pop_inc_10k_o', 'pop_inc_10k_15k_o', 'pop_inc_15k_25k_o', 'pop_inc_25k_35k_o', 'pop_inc_35k_50k_o', 'pop_inc_50k_65k_o', 'pop_inc_65k_75k_o', 'pop_inc_75k_o', 'geoid_y_o', 'est_pmiles_o', 'est_ptrp_o', 'est_vmiles_o', 'est_vtrp_o', 'total_pop_bts_o', 'households_bts_o', 'median_hh_inc_bts_o', 'hh_veh_0_o', 'hh_veh_1_o', 'hh_veh_2_more_o', 'pct_veh_0_o', 'pct_veh_1_o', 'pct_veh_2_more_o', 'hh_worker_1_o', 'pct_worker_1_o', 'hh_w

In [74]:
# pre processing flow_16
flow_16_acs_ma_all = flow_16_acs_ma.merge(us_tract_ma, left_on='Ofips', right_on='GEOID', how='inner', suffixes=('_od', '_o'))
flow_16_acs_ma_all = flow_16_acs_ma_all.merge(us_tract_ma, left_on='Dfips', right_on='GEOID', how='inner', suffixes=('_o', '_d'))
flow_16_acs_ma_all.head()
print("All columns are: ", [col for col in flow_16_acs_ma_all.columns])
print("Data shape (pre filering): ", flow_16_acs_ma_all.shape)

# turn to gpd with crs
flow_16_acs_ma_all = gpd.GeoDataFrame(flow_16_acs_ma_all, geometry='geometry_od',crs='EPSG:5070')

# add Euclidean distances & filter values
flow_16_acs_ma_all['euclidean_distance'] = flow_16_acs_ma_all.length

# filtering the self-flows
flow_16_acs_ma_all = flow_16_acs_ma_all.loc[flow_16_acs_ma_all['euclidean_distance']>0.0, :]
print("Data shape (filtering the self-loops)", flow_16_acs_ma_all.shape)

# drop ALL na
np.sum(flow_16_acs_ma_all.isna())
flow_16_acs_ma_all.dropna(inplace = True, how = 'any')
print("Data shape (after dropping all NAs)", flow_16_acs_ma_all.shape)


All columns are:  ['Ofips', 'Dfips', 'Ostfips', 'Octfips', 'Otrfips', 'Dstfips', 'Dctfips', 'Dtrfips', 'Flow', 'Moe', 'LenKM', 'ESTDIVMOE', 'geometry_od', 'STATEFP_o', 'COUNTYFP_o', 'TRACTCE_o', 'GEOID_o', 'NAME_o', 'NAMELSAD_o', 'MTFCC_o', 'FUNCSTAT_o', 'ALAND_o', 'AWATER_o', 'INTPTLAT_o', 'INTPTLON_o', 'layer_o', 'path_o', 'geoid_x_o', 'pop_o', 'households_o', 'male_o', 'male_ratio_o', 'female_o', 'female_ratio_o', 'median_age_o', 'white_o', 'black_o', 'asian_o', 'other_race_o', 'white_ratio_o', 'black_ratio_o', 'asian_ratio_o', 'other_race_ratio_o', 'income_o', 'pop_no_inc_o', 'pop_inc_10k_o', 'pop_inc_10k_15k_o', 'pop_inc_15k_25k_o', 'pop_inc_25k_35k_o', 'pop_inc_35k_50k_o', 'pop_inc_50k_65k_o', 'pop_inc_65k_75k_o', 'pop_inc_75k_o', 'geoid_y_o', 'est_pmiles_o', 'est_ptrp_o', 'est_vmiles_o', 'est_vtrp_o', 'total_pop_bts_o', 'households_bts_o', 'median_hh_inc_bts_o', 'hh_veh_0_o', 'hh_veh_1_o', 'hh_veh_2_more_o', 'pct_veh_0_o', 'pct_veh_1_o', 'pct_veh_2_more_o', 'hh_worker_1_o', 'pct

# Regressions

Q: Should I do the regression in the common vs. log space?

## Baseline Gravity Model

In [78]:
import statsmodels.api as sm

In [75]:
# flow 20
x = np.log(flow_20_march_week4_ma_all[['pop_o','pop_d','euclidean_distance']])
x = sm.add_constant(x)
y = np.log(flow_20_march_week4_ma_all['pop_flows'])

#  
mod = sm.OLS(y, x)
res=mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:              pop_flows   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     4690.
Date:                Sun, 18 Apr 2021   Prob (F-statistic):               0.00
Time:                        14:30:12   Log-Likelihood:                -66827.
No. Observations:               60482   AIC:                         1.337e+05
Df Residuals:                   60478   BIC:                         1.337e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  4.7190      0

In [76]:
# flow 16
x = np.log(flow_16_acs_ma_all[['pop_o','pop_d','euclidean_distance']])
x = sm.add_constant(x)
y = np.log(flow_16_acs_ma_all['Flow'])

#  
mod = sm.OLS(y, x)
res=mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   Flow   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     3472.
Date:                Sun, 18 Apr 2021   Prob (F-statistic):               0.00
Time:                        14:30:14   Log-Likelihood:            -1.0911e+05
No. Observations:               95478   AIC:                         2.182e+05
Df Residuals:                   95474   BIC:                         2.183e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  5.5409      0